# scraping home prices in egypt

"""
Scrap the data from aqarmap.com <br>
this data was collected in january 2026

data card:<br>
lat,lon -> location of the apartment (float)<br>
URL -> the link for the apartment in aqarmap site (object)<br>
Price -> total Price in pounds (float)<br>
Area -> Total Area for the apartment (float)<br>
Rooms -> total number of rooms (int)<br>
Bathrooms -> total number of bathrooms (int)<br>
Floor -> the apartment floor number (object)

"""


### Scraping the information from 1 house

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re

# We use a real User-Agent to avoid being blocked by the website's security
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

def parse_listing(url: str) -> dict:
    """
    Scrapes property details from an Aqarmap listing page.
    """
    try:
        resp = requests.get(url, headers=headers, timeout=15)
        resp.raise_for_status()
    except Exception as e:
        return {"url": url, "error": str(e)}

    soup = BeautifulSoup(resp.text, "html.parser")
    
    # Initialize data points
    price = lat = lon = area = rooms = bathrooms = floor = None

    # --- 1. PRICE & GEO (Extracted from JSON-LD scripts) ---
    # This is often the most accurate way to get raw numeric data
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string)
            # Handle both single objects and lists of objects
            items = data if isinstance(data, list) else [data]
            for item in items:
                # Get Price
                if "offers" in item and isinstance(item["offers"], dict):
                    price = item["offers"].get("price")
                # Get Coordinates
                if "geo" in item:
                    lat = item["geo"].get("latitude")
                    lon = item["geo"].get("longitude")
                elif item.get("@type") == "GeoCoordinates":
                    lat = item.get("latitude")
                    lon = item.get("longitude")
        except:
            continue

    # --- 2. MAIN ATTRIBUTES (Area, Rooms, Bathrooms) ---
    # These are found in the 'truncated-text' paragraphs near the icons
    attribute_items = soup.find_all('p', class_='text-body_1 truncated-text')
    for item in attribute_items:
        text = item.get_text(strip=True)
        # Match Rooms (غرف)
        if "غرف" in text:
            match = re.search(r'(\d+)', text)
            rooms = match.group(1) if match else None
        # Match Bathrooms (حمام)
        elif "حمام" in text:
            match = re.search(r'(\d+)', text)
            bathrooms = match.group(1) if match else None
        # Match Area (متر)
        elif "متر" in text or "m²" in text:
            match = re.search(r'(\d+)', text)
            area = match.group(1) if match else None

    # --- 3. TABLE DETAILS (Floor) ---
    # Target the specific grid structure you provided in the HTML snippet
    detail_rows = soup.find_all('div', class_=re.compile(r'flex px-1\.5x? py-2x'))
    for row in detail_rows:
        h4 = row.find('h4')
        if h4 and "الدور" in h4.get_text():
            value_span = row.find('span', class_='text-body_1')
            if value_span:
                floor = value_span.get_text(strip=True)
                break

    return {
        "url": url,
        "price": price,
        "area": area,
        "rooms": rooms,
        "bathrooms": bathrooms,
        "floor": floor,
        "lat": lat,
        "lon": lon,
    }

# --- TEST RUN ---
if __name__ == "__main__":
    # The listing URL you shared in your screenshots
    test_url = "https://aqarmap.com.eg/ar/listing/6627233-for-sale-cairo-el-sheikh-zayed-city-compounds-calm-residence-dcm/"
    
    data = parse_listing(test_url)
    
    print("--- Property Data ---")
    for key, value in data.items():
        print(f"{key.capitalize()}: {value}")

--- Property Data ---
Url: https://aqarmap.com.eg/ar/listing/6627233-for-sale-cairo-el-sheikh-zayed-city-compounds-calm-residence-dcm/
Price: 8660000
Area: 180
Rooms: 3
Bathrooms: 2
Floor: الارضي
Lat: 30.049397
Lon: 31.343136


### scraping the information for the houses in 1 page 

In [7]:
import requests
from bs4 import BeautifulSoup
import json
import re
import time
from urllib.parse import urljoin

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

def parse_listing_details(url):
    """function to get all details"""
    try:
        resp = requests.get(url, headers=headers, timeout=15)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        
        details = {
            "price": "N/A", "area": "N/A", "rooms": "N/A", 
            "bathrooms": "N/A", "floor": "N/A", "location": "N/A",
            "lat": "N/A", "lon": "N/A"
        }

        # 1. lat and lon from JSON-LD
        for script in soup.find_all("script", type="application/ld+json"):
            try:
                data = json.loads(script.string)
                items = data if isinstance(data, list) else [data]
                for item in items:
                    if "offers" in item: details["price"] = item["offers"].get("price")
                    if "geo" in item:
                        details["lat"] = item["geo"].get("latitude")
                        details["lon"] = item["geo"].get("longitude")
            except: continue

        # 2. location (State)
        location_tag = soup.find('p', class_=re.compile(r'text-caption.*truncated-text'))
        if location_tag:
            details["location"] = location_tag.get_text(strip=True)

        # 3. rooms, area, and bathrooms
        for item in soup.find_all('p', class_='text-body_1 truncated-text'):
            text = item.get_text(strip=True)
            if "غرف" in text: details["rooms"] = re.search(r'(\d+)', text).group(1)
            elif "حمام" in text: details["bathrooms"] = re.search(r'(\d+)', text).group(1)
            elif "متر" in text: details["area"] = re.search(r'(\d+)', text).group(1)

        # 4. floor
        for row in soup.find_all('div', class_=re.compile(r'flex px-1\.5x? py-2x')):
            h4 = row.find('h4')
            if h4 and "الدور" in h4.get_text():
                val = row.find('span', class_='text-body_1')
                if val: details["floor"] = val.get_text(strip=True)
        
        return details
    except Exception as e:
        print(f"Error parsing {url}: {e}")
        return None

def scrape_main_page(main_url):
    print(f"getting the urls: {main_url}...")
    response = requests.get(main_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    all_apartments = []
    # finding all urls
    links = soup.find_all('a', href=re.compile(r'/listing/'))
    seen_urls = set()

    for link in links:
        full_url = urljoin("https://aqarmap.com.eg", link['href'])
        if full_url not in seen_urls:
            seen_urls.add(full_url)
            
            print(f"checking : ...")
            extra_data = parse_listing_details(full_url)
            
            if extra_data:
                all_apartments.append({ "URL": full_url, **extra_data})
                
            time.sleep(1) # delay 

    return all_apartments

if __name__ == "__main__":
    search_url = "https://aqarmap.com.eg/ar/for-sale/apartment/"
    results = scrape_main_page(search_url)

    for apt in results:
        print(apt)

getting the urls: https://aqarmap.com.eg/ar/for-sale/apartment/...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
checking : ...
{'URL': 'https://aqarmap.com.eg/ar/listing/6495695-for-sale-cairo-new-cairo-el-banafsg-el-banafsag-1-tagan-st/', 'price': '5000000', 'area': '500', 'rooms': '4', 'bathrooms': '3', 'floor': '1', 'location': 'القاهرة الكبرى  /  شارع تاجان', 'lat': '30.049397', 'lon': '31.343136'}
{'URL': 'https://aqarmap.com.eg/ar/listing/6620952-for-sale-cairo-heliopolis-compounds-stoda-residence-il-cazar/', 'price': '5200000', 'area': '154', 'rooms': '3', 'bathrooms': '3', 'floor': '2', 'location': 'ستودا ريزيدنس - ال كازار', 'lat': '30.049397', 'lon': '31.343136'}
{'URL': 'https://aqarmap.com.eg/ar/listing/6625833-for-sale-cairo-el-maadi-sar

### scrapping number of pages then adding the data to a csv file 

In [ ]:
import math
import re
import time
import csv
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

# --- MATH HELPER ---
def tile_to_latlon(x, y, z):
    n = 2.0 ** z
    lon_deg = x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    lat_deg = math.degrees(lat_rad)
    return lat_deg, lon_deg

class AqarmapScraper:
    def __init__(self):
        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument("--start-maximized")
        # options.add_argument("--headless") # Uncomment if you want it to run in background
        
        self.driver = webdriver.Chrome(options=options)
        self.wait = WebDriverWait(self.driver, 15)
        self.results = []

    def get_coordinates(self):
        try:
            btn_xpath = "//button[contains(., 'اظهار الموقع')]"
            btn = self.driver.find_element(By.XPATH, btn_xpath)
            self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
            time.sleep(1)
            self.driver.execute_script("arguments[0].click();", btn)
            self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.leaflet-tile")))
            time.sleep(1.5)
            
            tiles = self.driver.find_elements(By.CSS_SELECTOR, "img.leaflet-tile")
            lat_sum, lon_sum, count = 0, 0, 0
            for tile in tiles:
                src = tile.get_attribute('src')
                match = re.search(r'/(\d+)/(\d+)/(\d+)\.png', src)
                if match:
                    z, x, y = map(int, match.groups())
                    lat, lon = tile_to_latlon(x, y, z)
                    lat_sum += lat
                    lon_sum += lon
                    count += 1
            if count > 0:
                return round((lat_sum / count) - 0.0015, 6), round((lon_sum / count) + 0.0025, 6)
        except: pass
        return "N/A", "N/A"

    def parse_listing(self, url):
        self.driver.get(url)
        time.sleep(2)
        data = {"URL": url, "Price": "N/A", "State": "N/A", "Rooms": "N/A", "Baths": "N/A", "Area": "N/A", "Lat": "N/A", "Lon": "N/A"}

        # JSON-LD Strategy
        try:
            scripts = self.driver.find_elements(By.XPATH, "//script[@type='application/ld+json']")
            for script in scripts:
                content = script.get_attribute('innerHTML')
                if '"Price"' in content or 'RealEstateListing' in content:
                    js = json.loads(content)
                    if 'offers' in js: data["Price"] = js['offers'].get('price', "N/A")
                    if 'address' in js: data["State"] = js['address'].get('addressLocality', "N/A")
        except: pass

        # Attributes Strategy
        try:
            features = self.driver.find_elements(By.CSS_SELECTOR, "p.text-body_1.truncated-text")
            for f in features:
                txt = f.text
                if "غرف" in txt: data["Rooms"] = re.search(r'\d+', txt).group()
                elif "حمام" in txt: data["Baths"] = re.search(r'\d+', txt).group()
                elif "متر" in txt: data["Area"] = re.search(r'\d+', txt).group()
        except: pass

        # Map Coordinates
        data["Lat"], data["Lon"] = self.get_coordinates()
        return data

    def run(self, base_url, start_page=1, end_page=500):
        try:
            for p in range(start_page, end_page + 1):
                print(f"--- Processing Page {p} ---")
                try:
                    self.driver.get(f"{base_url}?page={p}")
                    self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/listing/']")))
                    
                    # Robust URL extraction to prevent StaleElementReferenceException
                    urls = []
                    attempts = 0
                    while attempts < 3:
                        try:
                            found_links = self.driver.find_elements(By.CSS_SELECTOR, "a[href*='/listing/']")
                            urls = []
                            for l in found_links:
                                href = l.get_attribute('href')
                                if href: urls.append(href)
                            urls = list(dict.fromkeys(urls))
                            break
                        except StaleElementReferenceException:
                            attempts += 1
                            time.sleep(1)

                    for url in urls:
                        try:
                            print(f" -> Scraping: {url.split('/')[-2]}")
                            listing_data = self.parse_listing(url)
                            self.results.append(listing_data)
                            time.sleep(1)
                        except Exception as e:
                            print(f"Error on listing {url}: {e}")

                    self.save_data()
                
                except Exception as e:
                    print(f"Skipping Page {p} due to error: {e}")
                    continue
        finally:
            self.driver.quit()

    def save_data(self):
        if not self.results: return
        keys = self.results[0].keys()
        with open('aqarmap_data.csv', 'w', newline='', encoding='utf-8-sig') as f:
            dict_writer = csv.DictWriter(f, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(self.results)
        print(f"Saved {len(self.results)} total rows.")

if __name__ == "__main__":
    scraper = AqarmapScraper()
    target = "https://aqarmap.com.eg/ar/for-sale/apartment/"
    scraper.run(target, start_page=1, end_page=100)

### now we need to open the CSV file and add 2 new features, the floor and the finishing type


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import os
import re

# Headers to mimic a real browser session
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "ar,en-US;q=0.9,en;q=0.8",
}

def extract_features(url):
    """
    Extracts Floor and Finishing Type based on the HTML structure 
    seen in the browser inspector.
    
    """
    data = {"Floor": "N/A", "Finishing": "N/A"}
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        if response.status_code != 200:
            return data
        
        soup = BeautifulSoup(response.content, 'html.parser')

        # 1. ACCURATE FLOOR EXTRACTION 
        # We look for the h4 tag containing the word 'الدور'
        floor_label = soup.find("h4", string=re.compile(r"الدور"))
        if floor_label:
            # The value is in the span immediately following the h4
            floor_value = floor_label.find_next_sibling("span")
            if floor_value:
                data["Floor"] = floor_value.get_text(strip=True)

        # 2. FINISHING TYPE EXTRACTION
        # Usually found in the top summary icons or the details table
        finishing_keywords = ["تشطيب", "لوكس", "محارة", "طوب", "فرش"]
        # Search in the summary paragraph tags
        summary_features = soup.find_all("p", class_="text-body_1")
        for feature in summary_features:
            text = feature.get_text(strip=True)
            if any(key in text for key in finishing_keywords):
                data["Finishing"] = text
                break
        
        # Fallback: Search in the details table if not found in summary
        if data["Finishing"] == "N/A":
            finishing_label = soup.find("h4", string=re.compile(r"نوع التشطيب"))
            if finishing_label:
                finishing_value = finishing_label.find_next_sibling("span")
                if finishing_value:
                    data["Finishing"] = finishing_value.get_text(strip=True)

    except Exception as e:
        print(f"Error accessing {url}: {e}")
    
    return data

def run_enrichment(input_csv, output_csv):
    if not os.path.exists(input_csv):
        print(f"File {input_csv} not found!")
        return

    with open(input_csv, 'r', encoding='utf-8-sig') as f_in:
        reader = csv.DictReader(f_in)
        # Define new headers
        fieldnames = reader.fieldnames
        if "Floor" not in fieldnames: fieldnames.append("Floor")
        if "Finishing" not in fieldnames: fieldnames.append("Finishing")

        # Open output file
        with open(output_csv, 'w', newline='', encoding='utf-8-sig') as f_out:
            writer = csv.DictWriter(f_out, fieldnames=fieldnames)
            writer.writeheader()
            
            # Process each row
            rows = list(reader)
            print(f"Starting enrichment for {len(rows)} listings...")
            
            for i, row in enumerate(rows):
                url = row.get("URL")
                if url and url != "N/A":
                    print(f"[{i+1}/{len(rows)}] Scraping: {url.split('/')[-2]}")
                    new_data = extract_features(url)
                    row.update(new_data)
                
                # Write to file immediately
                writer.writerow(row)
                
                # FORCE SAVE: This ensures data is written to disk even if code crashes
                f_out.flush() 
                
                # Anti-ban sleep
                time.sleep(0.5)

    print(f"Done! Enriched data saved to: {output_csv}")

if __name__ == "__main__":
    input_file = 'aqarmap_final.csv'  # Your current file
    output_file = 'aqarmap_data3.csv' # The new saved file
    run_enrichment(input_file, output_file)

Starting enrichment for 1804 listings...
[1/1804] Scraping: 6495695-for-sale-cairo-new-cairo-el-banafsg-el-banafsag-1-tagan-st
[2/1804] Scraping: 6620952-for-sale-cairo-heliopolis-compounds-stoda-residence-il-cazar
[3/1804] Scraping: 6625833-for-sale-cairo-el-maadi-sarayat-el-maadi-street-256
[4/1804] Scraping: 6334217-for-sale-cairo-6th-of-october-bait-el-watan-bait-el-watan-el-asasy
[5/1804] Scraping: 6628064-for-sale-cairo-new-cairo-compounds-fifth-square
[6/1804] Scraping: 6603583-for-sale-cairo-new-cairo-new-narges
[7/1804] Scraping: 6400757-for-sale-cairo-new-cairo-compounds-dh-ykwn-ryzdns-styl-hwm
[8/1804] Scraping: 6609697-for-sale-cairo-new-cairo-el-banafsg-el-banafsag-3
[9/1804] Scraping: 6627035-for-sale-cairo-new-cairo-compounds-mwrd-fq-llttwyr
[10/1804] Scraping: 6583572-for-sale-cairo-new-cairo-compounds-village-garden-kattameya
[11/1804] Scraping: 6625071-for-sale-cairo-new-cairo-el-andalous-el-andalus-el-motamayez
[12/1804] Scraping: 6627233-for-sale-cairo-el-sheikh-zay

In [ ]:
# another improved version of the scraper to 
import os
import csv
import time
import math
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException


# --------------------------------------------------
# TILE → LAT/LON
# --------------------------------------------------
def tile_to_latlon(x, y, z):
    n = 2.0 ** z
    lon = x / n * 360.0 - 180.0
    lat = math.degrees(math.atan(math.sinh(math.pi * (1 - 2 * y / n))))
    return lat, lon


class AqarmapMergedScraper:

    def __init__(self, filename="aqarmap_final.csv"):
        self.filename = filename
        self.results = []

        options = Options()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument("--start-maximized")

        self.driver = webdriver.Chrome(options=options)
        self.wait = WebDriverWait(self.driver, 15)

    # --------------------------------------------------
    # WAIT FOR MAP JS (CRITICAL FIX)
    # --------------------------------------------------
    def wait_for_map_ready(self, timeout=8):
        script = """
        return (
            document.querySelector("div.angular-leaflet-map") ||
            document.querySelector("img.leaflet-tile") ||
            document.querySelector("path.leaflet-interactive")
        );
        """
        for _ in range(timeout * 2):
            try:
                if self.driver.execute_script(script):
                    return True
            except:
                pass
            time.sleep(0.5)
        return False

    # --------------------------------------------------
    # GET COORDINATES (ULTIMATE)
    # --------------------------------------------------
    def get_coordinates(self):
        try:

            placeholder_btn = self.driver.find_element(By.XPATH, "//a[contains(@ng-click, 'showListingMap')]")
            ng_click_text = placeholder_btn.get_attribute("ng-click")
            m = re.search(r"showListingMap$\s*([-0-9.]+),\s*([-0-9.]+)\s*$", ng_click_text)
            if m:
                return round(float(m.group(1)), 6), round(float(m.group(2)), 6)
        except:
            pass
        
        self.wait_for_map_ready()
        try:
            btns = self.driver.find_elements(By.XPATH, "//button[contains(., 'اظهار الموقع')] | //a[contains(., 'اظهر الموقع')]")
            if btns:
                btn = btns[0]
                self.driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
                time.sleep(0.5)
                self.driver.execute_script("arguments[0].click();", btn)
                time.sleep(2.5)
        except:
            pass

        # ✅ OPTION 1 — ng-init (EXACT)
        try:
            el = self.driver.find_element(By.CSS_SELECTOR, "div.angular-leaflet-map[ng-init]")
            ng = el.get_attribute("ng-init")
            m = re.search(r"initListingLocation$([-0-9.]+),\s*([-0-9.]+)$", ng)
            if m:
                return round(float(m.group(1)), 6), round(float(m.group(2)), 6)
        except:
            pass

        # ✅ OPTION 2 — ANGULAR SCOPE CENTER (🔥 NEW FIX)
        try:
            coords = self.driver.execute_script("""
            var el = document.querySelector("div.angular-leaflet-map");
            if (!el || !window.angular) return null;
            var scope = angular.element(el).scope();
            if (scope && scope.center && scope.center.lat) {
                return [scope.center.lat, scope.center.lng];
            }
            return null;
            """)
            if coords:
                return round(coords[0], 6), round(coords[1], 6)
        except:
            pass

        # ✅ OPTION 3 — LEAFLET CENTER
        try:
            coords = self.driver.execute_script("""
            for (var k in window) {
                var m = window[k];
                if (m && m._leaflet_id && m.getCenter) {
                    var c = m.getCenter();
                    if (c && c.lat) return [c.lat, c.lng];
                }
            }
            return null;
            """)
            if coords:
                return round(coords[0], 6), round(coords[1], 6)
        except:
            pass

        # ✅ OPTION 4 — SVG OVERLAY
        try:
            coords = self.driver.execute_script("""
            for (var k in window) {
                var m = window[k];
                if (m && m.layerPointToLatLng) {
                    var p = document.querySelector("path.leaflet-interactive");
                    if (!p) return null;
                    var b = p.getBBox();
                    var pt = L.point(b.x + b.width/2, b.y + b.height/2);
                    var ll = m.layerPointToLatLng(pt);
                    return [ll.lat, ll.lng];
                }
            }
            return null;
            """)
            if coords:
                return round(coords[0], 6), round(coords[1], 6)
        except:
            pass

        # ✅ OPTION 5 — TILE AVERAGE
        try:
            tiles = self.driver.find_elements(By.CSS_SELECTOR, "img.leaflet-tile")
            lat_sum = lon_sum = count = 0
            for t in tiles:
                src = t.get_attribute("src")
                m = re.search(r"/(\d+)/(\d+)/(\d+)\.png", src)
                if m:
                    z, x, y = map(int, m.groups())
                    lat, lon = tile_to_latlon(x, y, z)
                    lat_sum += lat
                    lon_sum += lon
                    count += 1
            if count:
                return round(lat_sum / count, 6), round(lon_sum / count, 6)
        except:
            pass

        return "N/A", "N/A"

    # --------------------------------------------------
    # MAP WITH RETRY
    # --------------------------------------------------
    def fetch_coordinates_with_retry(self, url, retries=2):
        for i in range(retries + 1):
            try:
                self.driver.get(url)
                time.sleep(3)
                lat, lon = self.get_coordinates()
                if lat != "N/A":
                    return lat, lon
            except:
                pass
        return "N/A", "N/A"

    # --------------------------------------------------
    # SCRAPE CARDS
    # --------------------------------------------------
    def scrape_cards_from_page(self):
        data = []
        cards = self.driver.find_elements(By.CSS_SELECTOR, "div.listing-card")

        for c in cards:
            try:
                link = c.find_elements(By.XPATH, ".//a[contains(@href,'/listing/')]")[0].get_attribute("href")
                price = c.find_element(By.CSS_SELECTOR, "span.text-title_4").text.strip()
                address = c.find_element(By.CSS_SELECTOR, "p.text-caption.truncated-text").get_attribute("title")

                def icon(cls):
                    try:
                        return c.find_element(By.XPATH, f".//i[contains(@class,'{cls}')]/following::p[1]").text
                    except:
                        return "N/A"

                data.append({
                    "URL": link,
                    "Price": price,
                    "Address": address,
                    "Area": icon("size-icon"),
                    "Rooms": icon("bedroom-icon"),
                    "Baths": icon("bathroom-icon")
                })
            except:
                continue
        return data

    # --------------------------------------------------
    # MAIN LOOP
    # --------------------------------------------------
    def run(self, base_url, start_page, end_page):
        for p in range(start_page, end_page + 1):
            print(f"\n--- Page {p} ---")
            try:
                self.driver.get(f"{base_url}?page={p}")
                self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.listing-card")))

                cards = self.scrape_cards_from_page()
                for item in cards:
                    item["Lat"], item["Lon"] = self.fetch_coordinates_with_retry(item["URL"])
                    self.results.append(item)

                self.save()
            except TimeoutException:
                continue

        self.driver.quit()

    # --------------------------------------------------
    # SAVE CSV
    # --------------------------------------------------
    def save(self):
        if not self.results:
            return
        write_header = not os.path.exists(self.filename)
        with open(self.filename, "a", newline="", encoding="utf-8-sig") as f:
            w = csv.DictWriter(f, fieldnames=self.results[0].keys())
            if write_header:
                w.writeheader()
            w.writerows(self.results)
        print(f"💾 Saved {len(self.results)} rows")
        self.results = []


# --------------------------------------------------
# RUN
# --------------------------------------------------
if __name__ == "__main__":
    scraper = AqarmapMergedScraper()
    scraper.run(
        "https://aqarmap.com.eg/ar/for-sale/apartment/",
        start_page=1606,
        end_page=2500
    )